In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import pandas as pd

class VehicleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    return accuracy, macro_f1

In [6]:
def main():
    # 设置随机种子
    torch.manual_seed(42)
    np.random.seed(42)
    
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")
    
    # 数据预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # 加载数据
    data_dir = "Data/Data"  # 修正数据路径
    image_paths = []
    labels = []
    
    # 遍历文件夹收集图片路径和标签
    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            # 从文件名中提取类别标签（1_xxx.png -> 0, 2_xxx.png -> 1, 3_xxx.png -> 2）
            class_id = int(filename.split('_')[0]) - 1  # 将1,2,3映射到0,1,2
            image_paths.append(os.path.join(data_dir, filename))
            labels.append(class_id)
    
    # 转换为numpy数组
    image_paths = np.array(image_paths)
    labels = np.array(labels)
    
    print(f"总样本数: {len(labels)}")
    print("类别分布:")
    for i in range(3):
        print(f"类别 {i+1}: {sum(labels == i)} 个样本")
    
    # 5折交叉验证
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    accuracies = []
    f1_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(image_paths, labels)):
        print(f"\nFold {fold + 1}")
        
        # 准备数据集
        train_dataset = VehicleDataset(
            [image_paths[i] for i in train_idx],
            [labels[i] for i in train_idx],
            transform=transform
        )
        val_dataset = VehicleDataset(
            [image_paths[i] for i in val_idx],
            [labels[i] for i in val_idx],
            transform=transform
        )
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        # 模型初始化
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 3)  # 3个类别
        model = model.to(device)
        
        # 损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        best_score = 0
        best_epoch = 0
        patience = 3
        no_improve = 0
        
        # 训练循环
        for epoch in range(30):  # 增加训练轮数到30
            train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
            accuracy, macro_f1 = evaluate(model, val_loader, device)
            
            # 计算当前fold的得分
            current_score = 0.7 * accuracy + 0.3 * macro_f1
            
            print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Score: {current_score:.4f}")
            
            # 早停策略
            if current_score > best_score:
                best_score = current_score
                best_epoch = epoch
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        
        # 记录最终结果
        final_accuracy, final_macro_f1 = evaluate(model, val_loader, device)
        accuracies.append(final_accuracy)
        f1_scores.append(final_macro_f1)
        
        print(f"\nFold {fold + 1} Final Results:")
        print(f"Accuracy: {final_accuracy:.4f}")
        print(f"Macro-F1: {final_macro_f1:.4f}")
        fold_score = 0.7 * final_accuracy + 0.3 * final_macro_f1
        print(f"Fold Score: {fold_score:.4f}")
    
    # 计算5折交叉验证的平均结果
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    print("\n最终五折交叉验证结果:")
    print(f"平均准确率: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"平均Macro-F1: {mean_f1:.4f} ± {std_f1:.4f}")
    
    # 计算最终平均得分
    final_score = 0.7 * mean_accuracy + 0.3 * mean_f1
    print(f"\n最终平均得分: {final_score:.4f}")

if __name__ == "__main__":
    main()

使用设备: cuda
总样本数: 1572
类别分布:
类别 1: 720 个样本
类别 2: 608 个样本
类别 3: 244 个样本

Fold 1


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\18794/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 11.4MB/s]



Epoch 1, Loss: 0.2514, Accuracy: 0.9524, Macro-F1: 0.9406, Score: 0.9489
Epoch 2, Loss: 0.1924, Accuracy: 0.9556, Macro-F1: 0.9584, Score: 0.9564
Epoch 2, Loss: 0.1924, Accuracy: 0.9556, Macro-F1: 0.9584, Score: 0.9564
Epoch 3, Loss: 0.0979, Accuracy: 0.9714, Macro-F1: 0.9708, Score: 0.9712
Epoch 3, Loss: 0.0979, Accuracy: 0.9714, Macro-F1: 0.9708, Score: 0.9712
Epoch 4, Loss: 0.0894, Accuracy: 0.9302, Macro-F1: 0.9289, Score: 0.9298
Epoch 4, Loss: 0.0894, Accuracy: 0.9302, Macro-F1: 0.9289, Score: 0.9298
Epoch 5, Loss: 0.1013, Accuracy: 0.9587, Macro-F1: 0.9489, Score: 0.9558
Epoch 5, Loss: 0.1013, Accuracy: 0.9587, Macro-F1: 0.9489, Score: 0.9558
Epoch 6, Loss: 0.0603, Accuracy: 0.9460, Macro-F1: 0.9412, Score: 0.9446
Early stopping at epoch 6
Epoch 6, Loss: 0.0603, Accuracy: 0.9460, Macro-F1: 0.9412, Score: 0.9446
Early stopping at epoch 6

Fold 1 Final Results:
Accuracy: 0.9492
Macro-F1: 0.9437
Fold Score: 0.9476

Fold 2

Fold 1 Final Results:
Accuracy: 0.9492
Macro-F1: 0.9437
Fold

c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2741, Accuracy: 0.8222, Macro-F1: 0.7966, Score: 0.8145
Epoch 2, Loss: 0.1161, Accuracy: 0.9365, Macro-F1: 0.9275, Score: 0.9338
Epoch 2, Loss: 0.1161, Accuracy: 0.9365, Macro-F1: 0.9275, Score: 0.9338
Epoch 3, Loss: 0.1143, Accuracy: 0.9270, Macro-F1: 0.8803, Score: 0.9130
Epoch 3, Loss: 0.1143, Accuracy: 0.9270, Macro-F1: 0.8803, Score: 0.9130
Epoch 4, Loss: 0.0701, Accuracy: 0.9746, Macro-F1: 0.9690, Score: 0.9729
Epoch 4, Loss: 0.0701, Accuracy: 0.9746, Macro-F1: 0.9690, Score: 0.9729
Epoch 5, Loss: 0.0401, Accuracy: 0.9683, Macro-F1: 0.9660, Score: 0.9676
Epoch 5, Loss: 0.0401, Accuracy: 0.9683, Macro-F1: 0.9660, Score: 0.9676
Epoch 6, Loss: 0.0161, Accuracy: 0.9651, Macro-F1: 0.9680, Score: 0.9660
Epoch 6, Loss: 0.0161, Accuracy: 0.9651, Macro-F1: 0.9680, Score: 0.9660
Epoch 7, Loss: 0.1183, Accuracy: 0.9111, Macro-F1: 0.8879, Score: 0.9041
Early stopping at epoch 7
Epoch 7, Loss: 0.1183, Accuracy: 0.9111, Macro-F1: 0.8879, Score: 0.9041
Early stopping at epoch 7

...

Epoch 1, Loss: 0.2536, Accuracy: 0.8917, Macro-F1: 0.8691, Score: 0.8849
Epoch 2, Loss: 0.1975, Accuracy: 0.9236, Macro-F1: 0.9087, Score: 0.9191
Epoch 2, Loss: 0.1975, Accuracy: 0.9236, Macro-F1: 0.9087, Score: 0.9191
Epoch 3, Loss: 0.0883, Accuracy: 0.9427, Macro-F1: 0.9432, Score: 0.9428
Epoch 3, Loss: 0.0883, Accuracy: 0.9427, Macro-F1: 0.9432, Score: 0.9428
Epoch 4, Loss: 0.0659, Accuracy: 0.8471, Macro-F1: 0.8677, Score: 0.8533
Epoch 4, Loss: 0.0659, Accuracy: 0.8471, Macro-F1: 0.8677, Score: 0.8533
Epoch 5, Loss: 0.0811, Accuracy: 0.9427, Macro-F1: 0.9179, Score: 0.9353
Epoch 5, Loss: 0.0811, Accuracy: 0.9427, Macro-F1: 0.9179, Score: 0.9353
Epoch 6, Loss: 0.0989, Accuracy: 0.9586, Macro-F1: 0.9551, Score: 0.9575
Epoch 6, Loss: 0.0989, Accuracy: 0.9586, Macro-F1: 0.9551, Score: 0.9575
Epoch 7, Loss: 0.1125, Accuracy: 0.9554, Macro-F1: 0.9426, Score: 0.9516
Epoch 7, Loss: 0.1125, Accuracy: 0.9554, Macro-F1: 0.9426, Score: 0.9516
Epoch 8, Loss: 0.1868, Accuracy: 0.9554, Macro-F1: 

c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2986, Accuracy: 0.8153, Macro-F1: 0.7925, Score: 0.8084
Epoch 2, Loss: 0.1428, Accuracy: 0.9427, Macro-F1: 0.9340, Score: 0.9401
Epoch 2, Loss: 0.1428, Accuracy: 0.9427, Macro-F1: 0.9340, Score: 0.9401
Epoch 3, Loss: 0.0547, Accuracy: 0.9650, Macro-F1: 0.9547, Score: 0.9619
Epoch 3, Loss: 0.0547, Accuracy: 0.9650, Macro-F1: 0.9547, Score: 0.9619
Epoch 4, Loss: 0.0771, Accuracy: 0.8662, Macro-F1: 0.8409, Score: 0.8586
Epoch 4, Loss: 0.0771, Accuracy: 0.8662, Macro-F1: 0.8409, Score: 0.8586
Epoch 5, Loss: 0.0699, Accuracy: 0.9618, Macro-F1: 0.9596, Score: 0.9611
Epoch 5, Loss: 0.0699, Accuracy: 0.9618, Macro-F1: 0.9596, Score: 0.9611
Epoch 6, Loss: 0.0600, Accuracy: 0.7516, Macro-F1: 0.6958, Score: 0.7348
Early stopping at epoch 6
Epoch 6, Loss: 0.0600, Accuracy: 0.7516, Macro-F1: 0.6958, Score: 0.7348
Early stopping at epoch 6

Fold 4 Final Results:
Accuracy: 0.7325
Macro-F1: 0.6767
Fold Score: 0.7158

Fold 5

Fold 4 Final Results:
Accuracy: 0.7325
Macro-F1: 0.6767
Fold

c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.3292, Accuracy: 0.9618, Macro-F1: 0.9573, Score: 0.9604
Epoch 2, Loss: 0.1881, Accuracy: 0.8153, Macro-F1: 0.8001, Score: 0.8107
Epoch 3, Loss: 0.0923, Accuracy: 0.9299, Macro-F1: 0.9335, Score: 0.9310
Epoch 4, Loss: 0.0775, Accuracy: 0.9459, Macro-F1: 0.9353, Score: 0.9427
Early stopping at epoch 4

Fold 5 Final Results:
Accuracy: 0.9459
Macro-F1: 0.9369
Fold Score: 0.9432

最终五折交叉验证结果:
平均准确率: 0.8988 ± 0.0840
平均Macro-F1: 0.8812 ± 0.1034

最终平均得分: 0.8935
